<a href="https://colab.research.google.com/github/KHB2937/MachineLearning/blob/main/MachineLearning/Kwak99/MLProject/MLNtest5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 나는 랜덤포레스트

In [ ]:
# 1. 나눔폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 2. 런타임 재시작 -> 런타임 / 런타임 다시 시작 (Ctrl + M .)
exit()

In [1]:
# 3. 기본 폰트를 나.바.고로 세팅
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [146]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import PolynomialFeatures
from math import sqrt
from sklearn.linear_model import Lasso
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [273]:

df_apt1 = pd.read_csv('https://github.com/jaiwon880/ML_Projiect/raw/main/Data/%EC%95%84%ED%8C%8C%ED%8A%B8_%EB%A7%A4%EB%A7%A4__%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_20230321134107_1%EC%9B%94.csv')
df_apt2 = pd.read_csv('https://raw.githubusercontent.com/jaiwon880/ML_Projiect/main/Data/%EC%95%84%ED%8C%8C%ED%8A%B8_%EB%A7%A4%EB%A7%A4__%EC%8B%A4%EA%B1%B0%EB%9E%98%EA%B0%80_20230321134120_2%EC%9B%94.csv')
df = pd.concat([df_apt1, df_apt2], ignore_index=True)
df


,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형
0,강원도 강릉시 견소동,84.9300,202301,4,"25,000",1997.0,중개거래
1,강원도 강릉시 견소동,84.9900,202301,5,"29,800",2005.0,중개거래
2,강원도 강릉시 견소동,84.9900,202301,25,"28,500",2005.0,중개거래
3,강원도 강릉시 교동,59.8600,202301,4,"20,000",2002.0,직거래
4,강원도 강릉시 교동,59.8400,202301,11,"25,000",2002.0,중개거래
...,...,...,...,...,...,...,...
52008,충청북도 충주시 호암동,84.9289,202302,16,"23,000",2007.0,중개거래
52009,충청북도 충주시 호암동,84.9289,202302,21,"23,000",2007.0,중개거래
52010,충청북도 충주시 호암동,84.6800,202302,2,"20,800",2002.0,중개거래
52011,충청북도 충주시 호암동,84.9500,202302,2,"40,000",2019.0,중개거래


## 전처리

In [274]:
df = pd.get_dummies(data=df, columns=['거래유형']) #pandas 데이터 셋에 그대로 인코딩 
df.head(5) 

,시군구,전용면적(㎡),계약년월,계약일,거래금액(만원),건축년도,거래유형_중개거래,거래유형_직거래
0,강원도 강릉시 견소동,84.93,202301,4,"25,000",1997.0,1,0
1,강원도 강릉시 견소동,84.99,202301,5,"29,800",2005.0,1,0
2,강원도 강릉시 견소동,84.99,202301,25,"28,500",2005.0,1,0
3,강원도 강릉시 교동,59.86,202301,4,"20,000",2002.0,0,1
4,강원도 강릉시 교동,59.84,202301,11,"25,000",2002.0,1,0


In [275]:
df.describe()

,전용면적(㎡),계약년월,계약일,건축년도,거래유형_중개거래,거래유형_직거래
count,52013.000000,52013.000000,52013.000000,52003.000000,52013.000000,52013.000000
mean,73.537838,202301.615442,15.228558,2006.086860,0.891335,0.108665
std,22.838342,0.486495,8.287285,10.634784,0.311222,0.311222
min,11.790000,202301.000000,1.000000,1962.000000,0.000000,0.000000
25%,59.820000,202301.000000,8.000000,1997.000000,1.000000,0.000000
50%,75.925800,202302.000000,15.000000,2006.000000,1.000000,0.000000
75%,84.950000,202302.000000,22.000000,2017.000000,1.000000,0.000000
max,290.997100,202302.000000,31.000000,2023.000000,1.000000,1.000000


In [276]:
df.drop(columns='거래유형_중개거래', inplace = True) # 중개거래 날리고
df.rename(columns = {'거래유형_직거래' : '거래유형'}, inplace = True) # 직거래는 1 중개는 0으로 나타냄
df["거래금액(만원)"] = df["거래금액(만원)"].str.replace(",", "").astype(int) # 거래금액 컬럼의 쉼표를 제거하고 정수형으로 변환
df["평당가"] = (df["거래금액(만원)"] / df["전용면적(㎡)"]*3.3) # 평당가 계산하여 새로운 컬럼 추가

df['건축년도'].unique()
df['거래유형'] = df['거래유형'].astype(int)

df.isna().sum() # NaN값 찾고 날리기기
df.dropna(inplace=True)

df['금리'] = df.apply(lambda x: 3.25 if ((x['계약년월']== 202301) & (x['계약일'] < 13)) else 3.5, axis=1)
df.drop(columns='시군구', inplace=True)
df.drop(columns='거래금액(만원)', inplace=True)
df['날짜'] = pd.to_datetime(df['계약년월'].astype(str) + df['계약일'].astype(str), format='%Y%m%d')
# 계약년월과 계약일 열 삭제 후 인덱스 설정
df = df.drop(['계약년월', '계약일'], axis=1)
df = df.set_index('날짜')

In [293]:
df

,전용면적(㎡),건축년도,거래유형,평당가,금리
날짜,,,,,
2023-01-04,84.9300,1997.0,0,971.388202,3.25
2023-01-05,84.9900,2005.0,0,1157.077303,3.25
2023-01-25,84.9900,2005.0,0,1106.600777,3.50
2023-01-04,59.8600,2002.0,1,1102.572670,3.25
2023-01-11,59.8400,2002.0,0,1378.676471,3.25
...,...,...,...,...,...
2023-02-16,84.9289,2007.0,0,893.688721,3.50
2023-02-21,84.9289,2007.0,0,893.688721,3.50
2023-02-02,84.6800,2002.0,0,810.581011,3.50


## 모델 테스트

In [307]:
X = df[['전용면적(㎡)', '건축년도', '거래유형', '금리']]
# X = df.drop("거래금액(만원)", axis=1)
y = df[['평당가']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)
y_train = y_train.values.ravel()



model = RandomForestRegressor(n_estimators=200, max_depth=6) # 랜덤 포레스트
model.fit(X_train, y_train)
predictions = model.predict(X_test)
score = model.score(X_test, y_test)
mean_squared_error(y_test, predictions)



1285549.4072557003

In [ ]:
# print(f"최적의 파라미터 : {grid_search.best_params_}") #그리드 서치 결과
# print(f"최고 성능 : {np.round(grid_search.best_score_ * 100, 2)}%")

In [308]:
mean_squared_error(y_test, predictions) ** 0.5

1133.8207121303176

In [309]:
mean_squared_error(y_test, predictions, squared=False)

1133.8207121303176

In [310]:
y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

In [311]:
mse = mean_squared_error(y_train, y_train_predict)
print("training set에서의 성능")
print("-----------------------")
print(sqrt(mse))

print("-----------------------")
mse = mean_squared_error(y_test, y_test_predict)
print("test set에서의 성능")
print("-----------------------")
print(sqrt(mse))

training set에서의 성능
-----------------------
1110.765747434115
-----------------------
test set에서의 성능
-----------------------
1133.8207121303176


In [312]:
predictions

array([1445.22751582, 1424.0157509 , 1235.41832503, ..., 2695.66141274,
       2004.32579067, 1853.67939594])

In [313]:
importances = model.feature_importances_

In [ ]:
indices_sorted = np.argsort(importances)

plt.figure()
plt.title("Feature importances")
plt.bar(range(len(importances)), importances[indices_sorted])
plt.xticks(range(len(importances)), X.columns[indices_sorted], rotation=90)
plt.show()

## 신규 데이터

In [316]:
new_x = [[110.1, 1994.0, 0, 3.25]]
new_df = pd.DataFrame(new_x, columns=['전용면적(㎡)', '건축년도', '거래유형', '금리'])
new_df = pd.DataFrame(new_x)

new_y = model.predict(new_df)
print("평당가 : {:.2f}만원".format(new_y[0]))


평당가 : 1608.12만원


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


## 배포

In [111]:
!pip install mlxtend -q

In [263]:
import joblib # pkl 세이브파일을 만들어주는 라이브러리

joblib.dump(model, 'Randomforest.pkl')

['Randomforest.pkl']

In [265]:
from google.colab import files

files.download('Randomforest.pkl') # 다운로드

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [114]:
model_from_joblib = joblib.load('/content/RandomForest.pkl')
model_from_joblib

RandomForestRegressor(max_depth=3, n_estimators=150)

In [218]:
import pandas as pd

# 새로운 데이터
new_x = [[90, 2022.0, 0, 900, 3.25]]
new_df = pd.DataFrame(new_x, columns=['전용면적(㎡)', '건축년도', '거래유형', '금리'])
# 전처리

new_df = pd.DataFrame(new_x)

# 모델 예측
new_y = model.predict(new_df)
print(new_y)

[23430.48579523]


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
